In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
torch.manual_seed(1)

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


# Create Train and Validation

In [37]:
X = []
Y = []
for p in tqdm(np.arange(0, 50), total = 50):
    X_p = torch.load('data_processed/1face_X_part' + str(p) + '.pt', map_location = device)
    Y_p = torch.load('data_processed/1face_Y_part' + str(p) + '.pt', map_location = device)
    X = X + X_p
    Y = Y + Y_p

100%|██████████| 50/50 [01:07<00:00,  1.54s/it]


In [38]:
len(X)

88209

In [39]:
len(Y)

88209

In [40]:
X = torch.stack(X)

In [67]:
X.shape

torch.Size([88209, 30, 512])

In [50]:
from torch.utils.data import TensorDataset, DataLoader
dataset = TensorDataset(X[0:88200], torch.from_numpy(np.array(Y[0:88200])))
train_data, val_data = torch.utils.data.random_split(dataset, [80000, 8200])
batch_size = 100

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

# Create Model

In [62]:
import torch.nn as nn

class DFDCNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.7):
        super(DFDCNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.7)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.float()
        
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [70]:
input_size = 512
output_size = 1
hidden_dim = 25
n_layers = 10

model = DFDCNet(input_size, output_size, hidden_dim, n_layers)
model.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
print(model)

DFDCNet(
  (lstm): LSTM(512, 25, num_layers=10, batch_first=True, dropout=0.7)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc): Linear(in_features=25, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [ ]:
epochs = 100
counter = 0
print_every = 1000
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './model_1face.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

Epoch: 2/100... Step: 1000... Loss: 0.623089... Val Loss: 0.623281
Validation loss decreased (inf --> 0.623281).  Saving model ...
Epoch: 3/100... Step: 2000... Loss: 0.492256... Val Loss: 0.483102
Validation loss decreased (0.623281 --> 0.483102).  Saving model ...
Epoch: 4/100... Step: 3000... Loss: 0.438843... Val Loss: 0.452579
Validation loss decreased (0.483102 --> 0.452579).  Saving model ...
Epoch: 5/100... Step: 4000... Loss: 0.517727... Val Loss: 0.451021
Validation loss decreased (0.452579 --> 0.451021).  Saving model ...
Epoch: 7/100... Step: 5000... Loss: 0.386208... Val Loss: 0.451296
Epoch: 8/100... Step: 6000... Loss: 0.408519... Val Loss: 0.451467
Epoch: 9/100... Step: 7000... Loss: 0.413138... Val Loss: 0.451641
Epoch: 10/100... Step: 8000... Loss: 0.413793... Val Loss: 0.451709
Epoch: 12/100... Step: 9000... Loss: 0.437181... Val Loss: 0.451636
Epoch: 13/100... Step: 10000... Loss: 0.474009... Val Loss: 0.451683
Epoch: 14/100... Step: 11000... Loss: 0.390873... Val L

# Reference
* https://github.com/gabrielloye/LSTM_Sentiment-Analysis/blob/master/main.ipynb